# 11. Data Science z Python 3.10. Identyfikacja klas zdjęć z fast.ai v2 i budowa aplikacji
## 🇬🇧 Data Science with Python 3.10. Optimizer, loss function and early stopping
#### 👨‍🏫 PhD Wojciech Oronowicz-Jaśkowiak
#### 🤖 https://github.com/aipogodzinach

### 1. Import bibliotek
#### 🇬🇧 Import libraries 

In [ ]:
!pip install fastbook
import fastbook
from fastbook import *
from fastai.vision.widgets import *
from IPython.core.pylabtools import figsize

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset_x2')
path = Path('/content/drive/My Drive/dataset_x2')

### 2. Brakuje danych treningowych?
#### 🇬🇧 Lack of data?

... zawsze możemy w tym celu skorzystać z zasobów Internetu i pobrać dane z Internetu. Uważaj jednak na słowa, które wpisujesz - w końcu pobierasz automatycznie dane z Internetu i nie chcesz mieć na swoim dysku Google dziwnych rzeczy ;)

In [ ]:
klasy = 'class_1', 'class_2' # w tym miejscu wpisujemy nazwę obiektu, który chcemy wyszukać np. 'pies' i 'kot'
path = Path('treningowe') # podajemy nazwę folderu, w którym znajdą się pliki
if not path.exists(): # folder zostanie dla nas stworzony, jeżeli jeszcze go nie mamy
  path.mkdir()
  for o in klasy:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    urls = search_images_ddg(f' {o} keyword', max_images=4) # w miejsce keyword podajemy dodatkowe słowo kluczowe do wyszukiwarki np. "zwierzaki"
    download_images(dest, urls=urls)

### 3. Trenowanie modelu
#### 🇬🇧 Model training

In [ ]:
classes = ['Pies', 'Kot']

In [ ]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5), # Określamy ilość materiałów treningowych oraz walidacyjnych, wielkość zdjęć, augmentację
    batch_tfms=aug_transforms(max_rotate=2.0)
)
dls = data.dataloaders(path, bs = 32, num_workers = 0)
# W fastai 2.0 dane przedstawiane są w formie DataBlock, to tutaj dokonujemy augmentacji danych

In [ ]:
dls.valid.show_batch(max_n=12, nrows=4)

In [ ]:
learn = vision_learner(dls, alexnet, metrics=accuracy)

In [ ]:
learn.fit(2)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.export(fname='model.pkl')